## Data Cleaning

#### Import modules

In [1]:
import os
import pandas as pd

#### Define paths and load data

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
PATH = 'data'

In [4]:
OUTPUT_PATH = 'outputs'

### Election results

#### Load data

In [5]:
results = pd.read_csv(os.path.join(PATH, 'HarvardData_countypres_2000-2020.csv'))
results = results[results['party'].isin(['DEMOCRAT', 'REPUBLICAN'])]
grpby_cols = ['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
              'candidate', 'party', 'totalvotes', 'version']
results = results.groupby(grpby_cols, as_index=False).sum()

In [6]:
results['pctvotes'] = results['candidatevotes'] / results['totalvotes']
results['office'] = 'PRESIDENT'  # for consistency

#### Pivot table

In [7]:
index_cols = ['year', 'state', 'state_po', 'county_name', 
              'county_fips', 'office']

In [8]:
pivoted_results = pd.pivot_table(results, index=index_cols, values=['pctvotes', 'candidatevotes'], columns=['party']).reset_index()
pivoted_results.columns.name = ''
pivoted_results.columns = [(x[0] + '_' + x[1]).lower() if x[1] != '' else x[0] for x in  pivoted_results.columns.values]
a = pivoted_results['candidatevotes_democrat'] > pivoted_results['candidatevotes_republican']
pivoted_results['winner'] = ['democrat' if x else 'republican' for x in a]
pivoted_results = pivoted_results[['year', 'state', 'county_name', 'county_fips', 'candidatevotes_democrat',
                                   'candidatevotes_republican', 'pctvotes_democrat', 'pctvotes_republican', 'winner']]

In [9]:
pivoted_results.head()

,year,state,county_name,county_fips,candidatevotes_democrat,candidatevotes_republican,pctvotes_democrat,pctvotes_republican,winner
0,2000,ALABAMA,AUTAUGA,1001.0,4942.0,11993.0,0.287192,0.696943,republican
1,2000,ALABAMA,BALDWIN,1003.0,13997.0,40872.0,0.247822,0.723654,republican
2,2000,ALABAMA,BARBOUR,1005.0,5188.0,5096.0,0.499086,0.490236,democrat
3,2000,ALABAMA,BIBB,1007.0,2710.0,4273.0,0.381636,0.601746,republican
4,2000,ALABAMA,BLOUNT,1009.0,4977.0,12667.0,0.276915,0.704779,republican


Export election results

In [10]:
for year in pivoted_results['year'].unique():
    
    df_temp = pivoted_results[pivoted_results['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'results_{year}.csv'), index=False)

### Unemployment data

In [11]:
unemployment = pd.read_excel(os.path.join(PATH, 'ERS_Unemployment.xlsx'), header=4)

In [12]:
rates_cols = ['Unemployment_rate_2000',
              'Unemployment_rate_2004',
              'Unemployment_rate_2008',
              'Unemployment_rate_2012',
              'Unemployment_rate_2016',
              'Unemployment_rate_2020']

In [13]:
area_cols = ['State', 'Area_name', 'FIPS_Code']

In [14]:
unemployment_ey = unemployment[area_cols + rates_cols]

In [15]:
unemployment_ey.sample(2)

,State,Area_name,FIPS_Code,Unemployment_rate_2000,Unemployment_rate_2004,Unemployment_rate_2008,Unemployment_rate_2012,Unemployment_rate_2016,Unemployment_rate_2020
533,GA,"Tattnall County, GA",13267,5.7,5.7,7.2,8.2,5.4,4.3
1354,MN,"Clay County, MN",27027,3.1,3.6,3.7,4.9,3.4,4.3


In [16]:
unemployment_ey_melted = pd.melt(unemployment_ey, 
                                 id_vars=area_cols, 
                                 value_vars=rates_cols, 
                                 value_name='unemployment',
                                 var_name='year')
unemployment_ey_melted['year'] = [x.split('_')[2] for x in unemployment_ey_melted['year']]
unemployment_ey_melted['year'] = unemployment_ey_melted['year'].astype(int)
unemployment_ey_melted.rename(columns={'FIPS_Code': 'county_fips'}, inplace=True)
unemployment_ey_melted = unemployment_ey_melted[['county_fips', 'year', 'unemployment']]

In [17]:
unemployment_ey_melted.sample()

,county_fips,year,unemployment
2717,48295,2000,3.4


Export unemployment data

In [18]:
for year in unemployment_ey_melted['year'].unique():
    
    df_temp = unemployment_ey_melted[unemployment_ey_melted['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'unemployment_{year}.csv'), index=False)

#### Population

In [19]:
population = pd.read_excel(os.path.join(PATH, 'ERS_PopulationEstimates.xls'), header=2)

In [20]:
population.sample()

,FIPStxt,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,POP_ESTIMATE_2010,POP_ESTIMATE_2011,POP_ESTIMATE_2012,POP_ESTIMATE_2013,POP_ESTIMATE_2014,POP_ESTIMATE_2015,POP_ESTIMATE_2016,POP_ESTIMATE_2017,POP_ESTIMATE_2018,POP_ESTIMATE_2019,N_POP_CHG_2010,N_POP_CHG_2011,N_POP_CHG_2012,N_POP_CHG_2013,N_POP_CHG_2014,N_POP_CHG_2015,N_POP_CHG_2016,N_POP_CHG_2017,N_POP_CHG_2018,N_POP_CHG_2019,Births_2010,Births_2011,Births_2012,Births_2013,Births_2014,Births_2015,Births_2016,Births_2017,Births_2018,Births_2019,Deaths_2010,Deaths_2011,Deaths_2012,Deaths_2013,Deaths_2014,Deaths_2015,Deaths_2016,Deaths_2017,Deaths_2018,Deaths_2019,NATURAL_INC_2010,NATURAL_INC_2011,NATURAL_INC_2012,NATURAL_INC_2013,NATURAL_INC_2014,NATURAL_INC_2015,NATURAL_INC_2016,NATURAL_INC_2017,NATURAL_INC_2018,NATURAL_INC_2019,INTERNATIONAL_MIG_2010,INTERNATIONAL_MIG_2011,INTERNATIONAL_MIG_2012,INTERNATIONAL_MIG_2013,INTERNATIONAL_MIG_2014,INTERNATIONAL_MIG_2015,INTERNATIONAL_MIG_2016,INTERNATIONAL_MIG_2017,INTERNATIONAL_MIG_2018,INTERNATIONAL_MIG_2019,DOMESTIC_MIG_2010,DOMESTIC_MIG_2011,DOMESTIC_MIG_2012,DOMESTIC_MIG_2013,DOMESTIC_MIG_2014,DOMESTIC_MIG_2015,DOMESTIC_MIG_2016,DOMESTIC_MIG_2017,DOMESTIC_MIG_2018,DOMESTIC_MIG_2019,NET_MIG_2010,NET_MIG_2011,NET_MIG_2012,NET_MIG_2013,NET_MIG_2014,NET_MIG_2015,NET_MIG_2016,NET_MIG_2017,NET_MIG_2018,NET_MIG_2019,RESIDUAL_2010,RESIDUAL_2011,RESIDUAL_2012,RESIDUAL_2013,RESIDUAL_2014,RESIDUAL_2015,RESIDUAL_2016,RESIDUAL_2017,RESIDUAL_2018,RESIDUAL_2019,GQ_ESTIMATES_BASE_2010,GQ_ESTIMATES_2010,GQ_ESTIMATES_2011,GQ_ESTIMATES_2012,GQ_ESTIMATES_2013,GQ_ESTIMATES_2014,GQ_ESTIMATES_2015,GQ_ESTIMATES_2016,GQ_ESTIMATES_2017,GQ_ESTIMATES_2018,GQ_ESTIMATES_2019,R_birth_2011,R_birth_2012,R_birth_2013,R_birth_2014,R_birth_2015,R_birth_2016,R_birth_2017,R_birth_2018,R_birth_2019,R_death_2011,R_death_2012,R_death_2013,R_death_2014,R_death_2015,R_death_2016,R_death_2017,R_death_2018,R_death_2019,R_NATURAL_INC_2011,R_NATURAL_INC_2012,R_NATURAL_INC_2013,R_NATURAL_INC_2014,R_NATURAL_INC_2015,R_NATURAL_INC_2016,R_NATURAL_INC_2017,R_NATURAL_INC_2018,R_NATURAL_INC_2019,R_INTERNATIONAL_MIG_2011,R_INTERNATIONAL_MIG_2012,R_INTERNATIONAL_MIG_2013,R_INTERNATIONAL_MIG_2014,R_INTERNATIONAL_MIG_2015,R_INTERNATIONAL_MIG_2016,R_INTERNATIONAL_MIG_2017,R_INTERNATIONAL_MIG_2018,R_INTERNATIONAL_MIG_2019,R_DOMESTIC_MIG_2011,R_DOMESTIC_MIG_2012,R_DOMESTIC_MIG_2013,R_DOMESTIC_MIG_2014,R_DOMESTIC_MIG_2015,R_DOMESTIC_MIG_2016,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019
2287,42007,PA,Beaver County,1.0,1.0,1.0,1.0,0.0,170539,170531,170630,170310,170056,169726,168977,168241,166758,165654,164582,163929,99.0,-320.0,-254.0,-330.0,-749.0,-736.0,-1483.0,-1104.0,-1072.0,-653.0,441.0,1698.0,1660.0,1737.0,1675.0,1663.0,1692.0,1651.0,1535.0,1506.0,418.0,1991.0,2091.0,2227.0,2085.0,2184.0,2144.0,2165.0,2176.0,2071.0,23.0,-293.0,-431.0,-490.0,-410.0,-521.0,-452.0,-514.0,-641.0,-565.0,0.0,15.0,20.0,26.0,20.0,13.0,29.0,16.0,13.0,9.0,95.0,-33.0,198.0,162.0,-328.0,-205.0,-1057.0,-601.0,-437.0,-86.0,95.0,-18.0,218.0,188.0,-308.0,-192.0,-1028.0,-585.0,-424.0,-77.0,-19.0,-9.0,-41.0,-28.0,-31.0,-23.0,-3.0,-5.0,-7.0,-11.0,3382.0,3382.0,3280.0,3299.0,3289.0,3254.0,3241.0,3210.0,3173.0,3196.0,3195.0,9.960697,9.754206,10.224203,9.890671,9.863056,10.101523,9.933456,9.296382,9.168643,11.679474,12.286774,13.108405,12.311671,12.953045,12.800038,13.026004,13.178454,12.608406,-1.718777,-2.532568,-2.884202,-2.421,-3.089989,-2.698516,-3.092548,-3.882072,-3.439763,0.087992,0.117521,0.153039,0.118098,0.077101,0.173135,0.096266,0.078732,0.054793,-0.193582,1.163453,0.953553,-1.9368,-1.215831,-6.310467,-3.615995,-2.646592,-0.523575,-0.10559,1.280974,1.106592,-1.818703,-1.138729,-6.137332,-3.519729,-2.567861,-0.46878

We obtained the rates for natural increase in population, international migration, domestic migration, and net migration for the years 2012, 2016, and we substituted the value of 2019 for 2020. 

In [21]:
years_population = ['2012', '2016', '2019']

In [22]:
rate_nat_inc_cols = ['R_NATURAL_INC_' + str(x) for x in years_population]
rate_int_mig_cols = ['R_INTERNATIONAL_MIG_' + str(x) for x in years_population]
rate_dom_mig_cols = ['R_DOMESTIC_MIG_' + str(x) for x in years_population]
rate_net_mig_cols = ['R_NET_MIG_' + str(x) for x in years_population]

In [23]:
population_dict = {'rate_natural_increase_population': rate_nat_inc_cols,
                   'rate_international_migration': rate_int_mig_cols,
                   'rate_domestic_migration': rate_dom_mig_cols,
                   'rate_net_migration': rate_net_mig_cols}

In [24]:
for key in population_dict.keys():
    
    cols = population_dict[key]
    temp2 = pd.melt(population[['FIPStxt'] + cols], 
                    id_vars=['FIPStxt'], 
                    value_vars=cols, 
                    value_name=key,
                    var_name='year')
    temp2['year'] = [x.split("_")[3] for x in temp2['year']]
    temp2['year'] = ['2020' if x == '2019' else x for x in temp2['year']]
    
    for year in temp2['year'].unique():
        df_temp = temp2[temp2['year'] == year]
        df_temp.to_csv(os.path.join(OUTPUT_PATH, f'{key}_{year}.csv'), index=False)
        

In [25]:
natural_inc_year = pd.melt(population[['FIPStxt'] + rate_nat_inc_cols], 
                           id_vars=['FIPStxt'], 
                           value_vars=rate_nat_inc_cols, 
                           value_name='natural_increase_in_pop',
                           var_name='year')

natural_inc_year['year'] = [x.split("_")[3] for x in natural_inc_year['year']]
natural_inc_year['year'] = ['2020' if x == '2019' else x for x in natural_inc_year['year']]

In [26]:
natural_inc_year

,FIPStxt,year,natural_increase_in_pop
0,0,2012,NaN
1,1000,2012,2.225976
2,1001,2012,0.998357
3,1003,2012,1.104318
4,1005,2012,0.513667
...,...,...,...
9814,72145,2020,NaN
9815,72147,2020,NaN
9816,72149,2020,NaN
9817,72151,2020,NaN


#### Results and unemployment merged